In [1]:
import pygame, sys, random
import numpy as np
import tensorflow as tf
import math
import matplotlib.pyplot as plt

num_episodes = 3601
dis = 0.99
rList = []

obstacleRadius = 10
agentRadius = 10

# get size of state and action from environment

boundaryPos = [50, 50]
boundaryLength = [200,200]
initPosAgent = [boundaryPos[0] + boundaryLength[0] / 2 + 100, boundaryPos[1] + boundaryLength[1] / 2]
initPosGoal =  [boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]

moveObstacles = False
action_size = 9
obsNumber = 4
state_size = 2
obsAngleUnit = 1

In [2]:
hidden_size = 2
learning_rate = 0.00002

# X = tf.placeholder(tf.float32,  [None, state_size], name = "input_x")
# W1 = tf.get_variable("W1", shape = [state_size, hidden_size], initializer = tf.contrib.layers.xavier_initializer())
# layer1 = tf.nn.tanh(tf.matmul(X, W1))
# W2 = tf.get_variable("W2", shape = [hidden_size, action_size], initializer = tf.contrib.layers.xavier_initializer())
X = tf.placeholder(tf.float32,  [None, state_size], name = "input_x")
W1 = tf.get_variable("W1", shape = [state_size, action_size], initializer = tf.contrib.layers.xavier_initializer())
# Qpred = tf.nn.tanh(tf.matmul(X, W1))

Qpred = tf.matmul(X,W1)

Y = tf.placeholder(shape  = [None, action_size], dtype = tf.float32)

loss = tf.reduce_sum(tf.square(Y - Qpred))
train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

In [3]:
def ckWall(xPos, yPos):
    flagWall = 0
    if(xPos < boundaryPos[0]):
        xPos = boundaryPos[0]
#         xPos = boundaryPos[0] + boundaryLength[0]
        flagWall = -1
    elif(xPos > boundaryPos[0] + boundaryLength[0]):
        xPos = boundaryPos[0] + boundaryLength[0]
#         xPos = boundaryPos[0]
        flagWall = -1
    if(yPos < boundaryPos[1]):
        yPos = boundaryPos[1]
#         yPos = boundaryPos[1] + boundaryLength[1]
        flagWall = -1
    elif(yPos > boundaryPos[1] + boundaryLength[1]):
        yPos = boundaryPos[1] + boundaryLength[1]
#         yPos = boundaryPos[1]
        flagWall = -1
    return [int(round(xPos)), int(round(yPos)), flagWall]

In [4]:
def takeAction(action):
    xAction = 0
    yAction = 0
    if action == 0:
        xAction = 1
    elif action == 1:
        xAction = 1
        yAction = 1
    elif action == 2:
        xAction = 1
        yAction = -1
    elif action == 3:
        xAction = -1
        yAction = 1
    elif action == 4:
        xAction = -1
    elif action == 5:
        xAction = -1
        yAction = -1
    elif action == 6:
        yAction = -1
    elif action == 7:
        yAction = 1
    elif action == 8:
        xAction = 0
        yAction = 0
        
    return [xAction, yAction]

In [5]:
def stateGenerator(intenPref, distPref):
    returnSum = [intenPref, distPref]
    returnSum = np.reshape(returnSum, [1, state_size])
#     print returnSum
    return returnSum

In [6]:
def rangeFinder(allObsPos, rangeCenter):
    allObsAgtDistance = [0 for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        allObsAgtDistance[i] = math.sqrt((allObsPos[i][0] - rangeCenter[0])**2 + (allObsPos[i][1] - rangeCenter[1])**2)
    index = np.argmin(allObsAgtDistance)
    return index

In [7]:
#display
pygame.init()
screen = pygame.display.set_mode([300,300])
screen.fill([200, 200, 200])

obsAngleIdx = 0
# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
posOffset = 60
obstaclePos[0][0] = int(initPosGoal[0] + posOffset) 
obstaclePos[0][1] = int(initPosGoal[1]) 
obstaclePos[1][0] = int(initPosGoal[0]) 
obstaclePos[1][1] = int(initPosGoal[1] + posOffset) 
obstaclePos[2][0] = int(initPosGoal[0] - posOffset) 
obstaclePos[2][1] = int(initPosGoal[1]) 
obstaclePos[3][0] = int(initPosGoal[0]) 
obstaclePos[3][1] = int(initPosGoal[1] - posOffset) 

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    for episode in range(num_episodes):
        done = False
        score = 0
        x = initPosAgent[0]
        y = initPosAgent[1]
        print("Episode ", episode, "Starts!")
        
        xMove = 0
        yMove = 0
        idx = rangeFinder(obstaclePos, [x,y])
        FeatureVec1 = (x - initPosGoal[0])**2 + (y - initPosGoal[1])**2
        FeatureVec2 = 1 / (0.2 + (x - obstaclePos[idx][0])**2 + (y - obstaclePos[idx][1])**2)
        state = stateGenerator(FeatureVec1, FeatureVec2)

        e = 1.0 / ((episode/400.0) + 1.0)
#         e = 0.1
#         e = 1.0 / (1.0 + episode / 40.0)
        rAll = 0
        step_count = 0

        while not done:
            step_count += 1
            Qinput = np.reshape(state, [1, state_size])
            Qs = sess.run(Qpred, feed_dict = {X: Qinput})
#             print Qs
            if np.random.rand(1) < e:
                action = random.randrange(0,8)
            else:
                action = np.argmax(Qs)
#             print action
            xMove = 0
            yMove = 0
            [xMove, yMove] = takeAction(action)

            x = x + xMove
            y = y + yMove
            
            wallFlag = 0
            collisionFlag = 0
            [x, y, wallFlag] = ckWall(x, y)
            pygame.draw.circle(screen, [100,100,255],[x,y],agentRadius,0)
            idx = rangeFinder(obstaclePos, [x,y])
            FeatureVec1 = (x - initPosGoal[0])**2 + (y - initPosGoal[1])**2
            FeatureVec2 = 1 / (0.2 + (x - obstaclePos[idx][0])**2 + (y - obstaclePos[idx][1])**2)
            next_state = stateGenerator(FeatureVec1, FeatureVec2)
            ## Draw Environment (Obstacle + Map)
            if(math.sqrt((x - initPosGoal[0])**2 + (y - initPosGoal[1])**2) <= obstacleRadius + agentRadius):
                print("Goal Reached!")
                reward = 20
                collisionFlag = 1
                done = True
            for i in range(0,obsNumber):
                pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
                if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) <= obstacleRadius + agentRadius:
                    print("Collision!", idx)
                    reward = -30
                    collisionFlag = -1
                    done = True
                    break
            if not done:
                reward = -1.0 * ((x - initPosGoal[0])**2 + (y - initPosGoal[1])**2) / 100000.0
#                 reward = 0

                Qinput1 = np.reshape(next_state, [1, state_size])
                Qs1 = sess.run(Qpred, feed_dict = {X: Qinput1})
                Qs[0,action] = reward + dis * np.max(Qs1)
            else:
                if collisionFlag == 1:
                    Qs[0, action] = 20
                    rList.append(1)
                elif collisionFlag == -1:
                    Qs[0, action] = -30
                    rList.append(0)
            
            sess.run(train, feed_dict={X: Qinput, Y: Qs})
            state = next_state
            
            if done:
                for i in range(0,obsNumber):
                    pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], obstacleRadius, 0)
            pygame.draw.circle(screen, [100,255,100], initPosGoal, 10, 1)
            pygame.draw.rect(screen, [255,100,100],[boundaryPos[0],boundaryPos[1], boundaryLength[0], boundaryLength[1]],1)
            pygame.display.flip()
            screen.fill([200,200,200])
        print("Episode : {}, Object Index: {}".format(episode, idx))

('Episode ', 0, 'Starts!')


KeyboardInterrupt: 

In [ ]:
print("Percent of successful episodes: " + str(sum(rList)/num_episodes) + "%")
plt.bar(range(len(rList)), rList, color = "white", width = 0.00001)
plt.show()